In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/combined_data.csv')

In [3]:
print('The features in the dataset are \n-------------------------------------------------------------------------------------')
print(df.columns)

The features in the dataset are 
-------------------------------------------------------------------------------------
Index(['UID', 'application', 'request_id', 'request_type', 'request_status',
       'account_name', 'entitlement', 'entitlement_value', 'approval_type',
       'approver', 'approved_on', 'approval_status', 'requested_on',
       'Unnamed: 0', 'employeeType', 'dept', 'div', 'hrOrganizationCode'],
      dtype='object')


In [4]:
#Removing unwanted columns from the dataset

columns = ['UID',
           'request_id',
           'request_type',
           'request_status',
           'account_name',
           'approval_type',
           'approver',
           'approved_on',
           'requested_on',
           'entitlement_value',
           'Unnamed: 0',
           'employeeType',
           'dept',
           'div',
           'hrOrganizationCode']

df.drop(columns, axis=1, inplace=True)

In [5]:
print('The shape of the dataset is -',df.shape)
print('-------------------------------------------------------------------------------------\n')

print('The features in the dataset are \n-------------------------------------------------------------------------------------')
print(df.columns)

The shape of the dataset is - (232281, 3)
-------------------------------------------------------------------------------------

The features in the dataset are 
-------------------------------------------------------------------------------------
Index(['application', 'entitlement', 'approval_status'], dtype='object')


In [6]:
df_approved = df.loc[df['approval_status']=='Approved']

print('The shape of the dataframe with approved requests is -',df_approved.shape)

The shape of the dataframe with approved requests is - (221414, 3)


In [7]:
#All requests grouped by application, entitlement and entitlement_value

df_gb = df.groupby(['entitlement','application'])
df_gb.first()

approval_status
entitlement    application                     
entitlement_0  Application_0           Rejected
               Application_101         Approved
               Application_102         Approved
               Application_103         Approved
               Application_1092        Approved
...                                         ...
entitlement_95 Application_265         Rejected
entitlement_96 Application_265         Rejected
entitlement_97 Application_265         Approved
entitlement_98 Application_265         Approved
entitlement_99 Application_265         Approved

[2209 rows x 1 columns]

In [8]:
#Approved requests grouped by entitlement

df_gb_approved = df_approved.groupby(['entitlement','application'])
df_gb_approved.first()

approval_status
entitlement    application                     
entitlement_0  Application_0           Approved
               Application_101         Approved
               Application_102         Approved
               Application_103         Approved
               Application_1092        Approved
...                                         ...
entitlement_95 Application_265         Approved
entitlement_96 Application_265         Approved
entitlement_97 Application_265         Approved
entitlement_98 Application_265         Approved
entitlement_99 Application_265         Approved

[2193 rows x 1 columns]

In [9]:
df_gb_approved_count = df_gb_approved.count()
df_gb_count = df_gb.count()

In [10]:
df_total = df_gb_count.reset_index()
df_approved = df_gb_approved_count.reset_index()

In [11]:
df_total.shape

(2209, 3)

In [12]:
df_approved.shape

(2193, 3)

#### To find difference in application-entitlement combinations in two datasets

In [13]:
df_temp1 = df_total.drop('approval_status', axis=1)
df_temp2 = df_approved.drop('approval_status' ,axis=1)

In [14]:
#Difference between two datasets

df_temp3 = pd.concat([df_temp2, df_temp1]).drop_duplicates(keep=False)
df_temp3

,entitlement,application
13,entitlement_0,Application_1159
517,entitlement_171,Application_563
518,entitlement_171,Application_564
538,entitlement_171,Application_588
539,entitlement_171,Application_589
554,entitlement_171,Application_615
573,entitlement_171,Application_655
630,entitlement_171,Application_751
911,entitlement_2,Application_1159
1055,entitlement_2,Application_221


In [15]:
filter1 = list(df_temp3['entitlement'])
filter2 = list(df_temp3['application'])

In [16]:
# Synchronizing the dataframes with each other

indexes=[]

for i in range(len(filter1)):
    post = filter1[i]
    app = filter2[i]
    index = df_temp3.loc[df_temp3.entitlement==post].loc[df_temp3.application==app].index[0]
    indexes.append(index)
    
indexes

[13,
 517,
 518,
 538,
 539,
 554,
 573,
 630,
 911,
 1055,
 1164,
 1354,
 1728,
 1734,
 2147,
 2168]

In [17]:
for x in indexes:
    df_total.drop(x, inplace=True)
    
df_total.shape

(2193, 3)

In [18]:
df_approved.shape

(2193, 3)

In [19]:
df_total.reset_index(inplace=True)
df_approved.reset_index(inplace=True)

In [20]:
df_total.drop('index', axis=1, inplace=True)
df_approved.drop('index', axis=1, inplace=True)

In [21]:
print('The features of the total dataset are -\n',df_total.columns)
print('-------------------------------------------------------------------------------------\n')

print('The features in the dataset with approved requests are \n-------------------------------------------------------------------------------------')
print(df_approved.columns)

The features of the total dataset are -
 Index(['entitlement', 'application', 'approval_status'], dtype='object')
-------------------------------------------------------------------------------------

The features in the dataset with approved requests are 
-------------------------------------------------------------------------------------
Index(['entitlement', 'application', 'approval_status'], dtype='object')


In [22]:
df_total.rename(columns={'approval_status':'total_requests'}, inplace=True)

In [23]:
approved = df_approved['approval_status']
df_total['approved'] = approved

In [24]:
total = df_total['total_requests']
per = (approved/total*100).round(2)

df_total['percentage'] = per

In [25]:
df_total.head(20)

,entitlement,application,total_requests,approved,percentage
0,entitlement_0,Application_0,489,420,85.89
1,entitlement_0,Application_101,21,21,100.00
2,entitlement_0,Application_102,4,4,100.00
3,entitlement_0,Application_103,74,72,97.30
4,entitlement_0,Application_1092,2,1,50.00
5,entitlement_0,Application_1093,779,766,98.33
6,entitlement_0,Application_1100,22,21,95.45
7,entitlement_0,Application_111,3,3,100.00
8,entitlement_0,Application_1111,2,2,100.00
9,entitlement_0,Application_1115,2,2,100.00


In [26]:
recommendations = df_total[['entitlement','application', 'percentage']].loc[df_total.total_requests>=10].loc[df_total.percentage>=80]

In [28]:
def predict(post):
    recommendations = df_total[['entitlement','application','percentage']].loc[df_total.entitlement==post]
    if len(recommendations)>0:
        return recommendations
    else:
        return 'No recommendations for this entitlement are available yet..!!!'

In [30]:
output = predict('entitlement_9')
output

,entitlement,application,percentage
2176,entitlement_9,Application_1094,96.89
2177,entitlement_9,Application_1109,97.37
2178,entitlement_9,Application_1126,91.27
2179,entitlement_9,Application_1183,89.52
2180,entitlement_9,Application_13,94.00
2181,entitlement_9,Application_194,100.00
2182,entitlement_9,Application_229,85.98
